In [1]:
import pandas as pd
import numpy as np

## Data Cleaning and Prep

#### Dataset 1: Streeteasy

In [2]:
inventorystudio = pd.read_csv('../data/raw/streeteasy/rentalInventory_Studio.csv')
inventory1 = pd.read_csv('../data/raw/streeteasy/rentalInventory_OneBd.csv')
inventory2 = pd.read_csv('../data/raw/streeteasy/rentalInventory_TwoBd.csv')
inventory3 = pd.read_csv('../data/raw/streeteasy/rentalInventory_ThreePlusBd.csv')
 
rentstudio = pd.read_csv('../data/raw/streeteasy/medianAskingRent_Studio.csv')
rent1 = pd.read_csv('../data/raw/streeteasy/medianAskingRent_OneBd.csv')
rent2 = pd.read_csv('../data/raw/streeteasy/medianAskingRent_TwoBd.csv')
rent3 = pd.read_csv('../data/raw/streeteasy/medianAskingRent_ThreePlusBd.csv')

Filter for neighborhood data and add bedroom feature. There are 177 neighborhoods but we will remove several neighborhoods with many NaN values and also impute some values.

In [3]:
rentstudio = rentstudio[rentstudio.areaType=='neighborhood']
rent1 = rent1[rent1.areaType == 'neighborhood']
rent2 = rent2[rent2.areaType == 'neighborhood']
rent3 = rent3[rent3.areaType == 'neighborhood']

rentstudio['bd'] = 0
rent1['bd'] = 1
rent2['bd'] = 2
rent3['bd'] = 3

In [4]:
inventory1.head(3)

,areaName,Borough,areaType,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,...,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10
0,All Downtown,Manhattan,submarket,1645.0,1760.0,1763.0,1840.0,1744.0,1752.0,1628.0,...,1976.0,1846.0,1675.0,1730.0,2799.0,3855.0,5014.0,5474.0,5439.0,5284.0
1,All Midtown,Manhattan,submarket,1410.0,1462.0,1489.0,1593.0,1416.0,1384.0,1386.0,...,1587.0,1445.0,1256.0,1257.0,1716.0,2378.0,3098.0,3445.0,3444.0,3499.0
2,All Upper East Side,Manhattan,submarket,1064.0,1158.0,1157.0,1094.0,1093.0,1137.0,1110.0,...,887.0,822.0,756.0,671.0,1072.0,1471.0,1970.0,2243.0,2297.0,2392.0


In [5]:
rent3.head(3)

,areaName,Borough,areaType,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,...,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,bd
5,Astoria,Queens,neighborhood,2000.0,2150.0,2100.0,2050.0,2000.0,2150.0,NaN,...,2950.0,2985.0,3000.0,3000.0,2900.0,2900.0,2900.0,2800.0,2700.0,3
6,Auburndale,Queens,neighborhood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
7,Bath Beach,Brooklyn,neighborhood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [6]:
inventorystudio= inventorystudio[inventorystudio.areaType == 'neighborhood']
inventory1=inventory1[inventory1.areaType == 'neighborhood']
inventory2=inventory2[inventory2.areaType == 'neighborhood']
inventory3=inventory3[inventory3.areaType == 'neighborhood']


inventorystudio['bd']=0
inventory1['bd']=1
inventory2['bd']=2
inventory3['bd']=3

Merge inventory and rent data sets.

In [7]:
rent12 = pd.concat([rent1, rent2])
rent3s= pd.concat([rent3,rentstudio])
rent_full=pd.concat([rent12,rent3s])

inventory12=pd.concat([inventory1,inventory2])
inventory3s=pd.concat([inventory3,inventorystudio])
inventory_full=pd.concat([inventory12,inventory3s])

#### Matching neighborhood to zipcode

Manual mapping done in the nyc_zip_neighborhood file below. Join to match neighborhood to zipcode.

In [8]:
zip2n=pd.read_csv('../data/raw/streeteasy/nyc_zip_neighorbood.csv')
zip2n=zip2n.iloc[:, :-1]

In [9]:
zip2n.head(3)

,zipcode,borough,neighborhood
0,10001,Manhattan,Chelsea
1,10002,Manhattan,Lower East Side
2,10003,Manhattan,Flatiron


In [10]:
rent_full = zip2n.merge(rent_full, left_on='neighborhood',right_on='areaName').drop(['areaName','Borough'],axis=1)
inventory_full = zip2n.merge(inventory_full, left_on='neighborhood',right_on='areaName').drop(['areaName','Borough'],axis=1)

In [11]:
rent_full.head(10)

,zipcode,borough,neighborhood,areaType,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,...,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,bd
0,10001,Manhattan,Chelsea,neighborhood,3150.0,3270.0,3050.0,2993.0,3020.0,3000.0,...,4193.0,4175.0,3999.0,3850.0,3693.0,3500.0,3400.0,3308.0,3381.0,1
1,10001,Manhattan,Chelsea,neighborhood,4900.0,5500.0,5500.0,5495.0,5473.0,5498.0,...,6184.0,6170.0,5940.0,5478.0,5031.0,4975.0,4963.0,5076.0,5000.0,2
2,10001,Manhattan,Chelsea,neighborhood,8500.0,10725.0,9750.0,8900.0,6300.0,6300.0,...,12500.0,13250.0,9700.0,6995.0,6450.0,6200.0,5995.0,6600.0,6600.0,3
3,10001,Manhattan,Chelsea,neighborhood,2400.0,2200.0,2200.0,2295.0,2463.0,2425.0,...,3073.0,3075.0,2963.0,2850.0,2704.0,2554.0,2500.0,2465.0,2400.0,0
4,10011,Manhattan,Chelsea,neighborhood,3150.0,3270.0,3050.0,2993.0,3020.0,3000.0,...,4193.0,4175.0,3999.0,3850.0,3693.0,3500.0,3400.0,3308.0,3381.0,1
5,10011,Manhattan,Chelsea,neighborhood,4900.0,5500.0,5500.0,5495.0,5473.0,5498.0,...,6184.0,6170.0,5940.0,5478.0,5031.0,4975.0,4963.0,5076.0,5000.0,2
6,10011,Manhattan,Chelsea,neighborhood,8500.0,10725.0,9750.0,8900.0,6300.0,6300.0,...,12500.0,13250.0,9700.0,6995.0,6450.0,6200.0,5995.0,6600.0,6600.0,3
7,10011,Manhattan,Chelsea,neighborhood,2400.0,2200.0,2200.0,2295.0,2463.0,2425.0,...,3073.0,3075.0,2963.0,2850.0,2704.0,2554.0,2500.0,2465.0,2400.0,0
8,10002,Manhattan,Lower East Side,neighborhood,1995.0,1975.0,1900.0,1850.0,2050.0,2100.0,...,3650.0,3000.0,2900.0,2760.0,2650.0,2500.0,2495.0,2400.0,2350.0,1
9,10002,Manhattan,Lower East Side,neighborhood,2625.0,2698.0,2600.0,2600.0,2600.0,2600.0,...,4200.0,3999.0,3725.0,3400.0,3199.0,2995.0,2916.0,2800.0,2895.0,2


#### Dealing with missing data

In [12]:
rent_full.describe()

,zipcode,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,...,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,bd
count,872.000000,314.000000,312.000000,316.000000,318.000000,307.000000,302.000000,305.000000,288.000000,287.000000,...,543.000000,524.000000,476.000000,533.000000,559.000000,580.000000,586.000000,569.000000,571.000000,872.000000
mean,10812.747706,3214.522293,3295.192308,3211.110759,3196.924528,3317.247557,3402.099338,3355.622951,3500.940972,3607.648084,...,3392.158379,3437.175573,3369.077731,3160.041276,3063.164580,2970.675862,2902.056314,2883.789104,2861.786340,1.500000
std,568.570642,2180.060749,2434.657188,2223.490907,2258.726776,2424.725891,2616.723317,2475.015579,2633.691057,2642.653237,...,2467.448983,2476.121511,2107.008078,1957.696658,1872.072997,1774.866818,1739.564492,1736.837991,1794.153655,1.118676
min,10001.000000,950.000000,950.000000,950.000000,950.000000,900.000000,900.000000,900.000000,900.000000,900.000000,...,1400.000000,1410.000000,1400.000000,1400.000000,1485.000000,1495.000000,1450.000000,1450.000000,1450.000000,0.000000
25%,10280.000000,1800.000000,1818.750000,1800.000000,1853.250000,1895.000000,1895.000000,1858.000000,1975.000000,1995.000000,...,2081.500000,2188.750000,2228.000000,2100.000000,1994.000000,1990.000000,1963.000000,1928.000000,1895.000000,0.750000
50%,11203.000000,2500.000000,2400.000000,2350.000000,2470.000000,2500.000000,2500.000000,2548.000000,2600.000000,2700.000000,...,2600.000000,2600.000000,2700.000000,2585.000000,2500.000000,2500.000000,2440.000000,2399.000000,2350.000000,1.500000
75%,11354.000000,3809.750000,3825.000000,3800.000000,3625.000000,3981.500000,3937.500000,3895.000000,3962.500000,4250.000000,...,3500.000000,3500.000000,3889.000000,3483.000000,3409.000000,3103.250000,3000.000000,3000.000000,2958.500000,2.250000
max,11697.000000,11750.000000,17000.000000,14225.000000,13725.000000,15000.000000,16000.000000,15000.000000,15000.000000,15000.000000,...,21000.000000,22000.000000,16558.000000,19000.000000,19000.000000,17654.000000,20308.000000,16500.000000,18250.000000,3.000000


In [13]:
inventory_full.describe()

,zipcode,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,...,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,bd
count,872.000000,572.000000,579.000000,574.000000,573.000000,542.000000,532.000000,538.000000,528.000000,534.000000,...,788.000000,778.000000,744.000000,756.000000,773.000000,780.000000,777.000000,769.000000,774.000000,872.000000
mean,10812.747706,82.057692,83.542314,85.398955,86.356021,85.998155,90.590226,85.033457,81.922348,72.168539,...,88.779188,83.097686,82.513441,133.518519,184.294955,235.720513,262.326898,264.284785,259.622739,1.500000
std,568.570642,175.696917,183.232725,189.278623,192.100204,187.913492,195.601495,184.118249,176.236826,157.792849,...,142.736843,133.264808,131.434470,215.222468,302.489902,397.282416,441.620850,444.412684,443.867639,1.118676
min,10001.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,10280.000000,3.000000,3.000000,3.000000,3.000000,2.000000,3.000000,3.000000,3.000000,3.000000,...,6.000000,6.000000,6.000000,6.000000,8.000000,9.000000,10.000000,9.000000,9.000000,0.750000
50%,11203.000000,13.000000,13.000000,16.000000,14.000000,18.000000,17.500000,14.500000,14.000000,11.000000,...,24.500000,22.500000,24.000000,31.000000,39.000000,46.000000,51.000000,56.000000,53.000000,1.500000
75%,11354.000000,61.000000,66.000000,61.000000,58.000000,72.000000,67.500000,66.000000,63.000000,57.000000,...,99.000000,91.000000,101.000000,151.250000,204.000000,275.000000,307.000000,298.000000,289.000000,2.250000
max,11697.000000,1064.000000,1158.000000,1157.000000,1148.000000,1116.000000,1137.000000,1110.000000,1110.000000,990.000000,...,822.000000,756.000000,755.000000,1158.000000,1612.000000,2122.000000,2274.000000,2340.000000,2392.000000,3.000000


In [14]:
rent_slice=rent_full.iloc[:,np.r_[0:3, 105:135]]
rent=rent_slice.dropna(thresh=rent_slice.shape[1]-5,axis=0)
len(rent)

511

In [15]:
inventory_slice=inventory_full.iloc[:,np.r_[0:3, 105:135]]
inv=inventory_slice.dropna(thresh=inventory_slice.shape[1]-5,axis=0)
len(inv)

722

In [16]:
inv.reset_index(inplace=True)
inv = inv.drop('index',axis=1)

rent.reset_index(inplace=True)
rent = rent.drop('index',axis=1)

In [17]:
rent.iloc[:,3:134]

,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,...,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,bd
0,3872.0,3848.0,4033.0,4165.0,3988.0,3995.0,4003.0,4179.0,4295.0,4202.0,...,4193.0,4175.0,3999.0,3850.0,3693.0,3500.0,3400.0,3308.0,3381.0,1
1,5800.0,5595.0,5500.0,5997.0,5927.0,5975.0,6193.0,6200.0,6056.0,5954.0,...,6184.0,6170.0,5940.0,5478.0,5031.0,4975.0,4963.0,5076.0,5000.0,2
2,6708.0,7000.0,7250.0,7125.0,7500.0,10600.0,9998.0,10375.0,10250.0,10250.0,...,12500.0,13250.0,9700.0,6995.0,6450.0,6200.0,5995.0,6600.0,6600.0,3
3,2834.0,2850.0,2875.0,2875.0,2829.0,2875.0,2963.0,2900.0,3042.0,3000.0,...,3073.0,3075.0,2963.0,2850.0,2704.0,2554.0,2500.0,2465.0,2400.0,0
4,3872.0,3848.0,4033.0,4165.0,3988.0,3995.0,4003.0,4179.0,4295.0,4202.0,...,4193.0,4175.0,3999.0,3850.0,3693.0,3500.0,3400.0,3308.0,3381.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,2200.0,1925.0,1900.0,2000.0,2060.0,2100.0,2008.0,2113.0,1995.0,1948.0,...,NaN,NaN,NaN,2175.0,1999.0,1999.0,2100.0,2125.0,2100.0,2
507,2200.0,2300.0,2800.0,2800.0,2350.0,2299.0,2295.0,2297.0,2300.0,2313.0,...,2499.0,2499.0,2500.0,2400.0,3150.0,2500.0,2950.0,NaN,2760.0,3
508,1599.0,1650.0,1600.0,1650.0,1600.0,1670.0,1695.0,1700.0,1695.0,1650.0,...,1600.0,1663.0,1639.0,1639.0,1750.0,1603.0,1614.0,1626.0,1699.0,1
509,2200.0,1925.0,1900.0,2000.0,2060.0,2100.0,2008.0,2113.0,1995.0,1948.0,...,NaN,NaN,NaN,2175.0,1999.0,1999.0,2100.0,2125.0,2100.0,2


Use KNN to impute. From sklearn documentation:

_Each sample’s missing values are imputed using the mean value from n_neighbors nearest neighbors found in the training set. Two samples are close if the features that neither is missing are close._

In [18]:
from sklearn.impute import KNNImputer

In [19]:
imputer_rent = KNNImputer(n_neighbors=2)
imputed_rent = pd.DataFrame(imputer_rent.fit_transform(rent.iloc[:,3:134]))

imputer_inv = KNNImputer(n_neighbors=2)
imputed_inv = pd.DataFrame(imputer_inv.fit_transform(inv.iloc[:,3:134]))

In [20]:
rent_clean = pd.concat([rent.iloc[:,:3],imputed_rent],axis=1,ignore_index=True)
rent_clean.columns = rent.columns

inv_clean = pd.concat([inv.iloc[:,:3],imputed_inv],axis=1,ignore_index=True)
inv_clean.columns = inv.columns

In [21]:
rent_clean

,zipcode,borough,neighborhood,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,...,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,bd
0,10001,Manhattan,Chelsea,3872.0,3848.0,4033.0,4165.0,3988.0,3995.0,4003.0,...,4193.0,4175.0,3999.0,3850.0,3693.0,3500.0,3400.0,3308.0,3381.0,1.0
1,10001,Manhattan,Chelsea,5800.0,5595.0,5500.0,5997.0,5927.0,5975.0,6193.0,...,6184.0,6170.0,5940.0,5478.0,5031.0,4975.0,4963.0,5076.0,5000.0,2.0
2,10001,Manhattan,Chelsea,6708.0,7000.0,7250.0,7125.0,7500.0,10600.0,9998.0,...,12500.0,13250.0,9700.0,6995.0,6450.0,6200.0,5995.0,6600.0,6600.0,3.0
3,10001,Manhattan,Chelsea,2834.0,2850.0,2875.0,2875.0,2829.0,2875.0,2963.0,...,3073.0,3075.0,2963.0,2850.0,2704.0,2554.0,2500.0,2465.0,2400.0,0.0
4,10011,Manhattan,Chelsea,3872.0,3848.0,4033.0,4165.0,3988.0,3995.0,4003.0,...,4193.0,4175.0,3999.0,3850.0,3693.0,3500.0,3400.0,3308.0,3381.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,11694,Queens,Rockaway All,2200.0,1925.0,1900.0,2000.0,2060.0,2100.0,2008.0,...,2100.0,2200.0,2150.0,2175.0,1999.0,1999.0,2100.0,2125.0,2100.0,2.0
507,11694,Queens,Rockaway All,2200.0,2300.0,2800.0,2800.0,2350.0,2299.0,2295.0,...,2499.0,2499.0,2500.0,2400.0,3150.0,2500.0,2950.0,2695.0,2760.0,3.0
508,11697,Queens,Rockaway All,1599.0,1650.0,1600.0,1650.0,1600.0,1670.0,1695.0,...,1600.0,1663.0,1639.0,1639.0,1750.0,1603.0,1614.0,1626.0,1699.0,1.0
509,11697,Queens,Rockaway All,2200.0,1925.0,1900.0,2000.0,2060.0,2100.0,2008.0,...,2100.0,2200.0,2150.0,2175.0,1999.0,1999.0,2100.0,2125.0,2100.0,2.0


In [22]:
rent_clean=rent_clean.groupby(['zipcode','bd']).mean()
inv_clean=inv_clean.groupby(['zipcode','bd']).mean()

In [23]:
rent_clean.reset_index(inplace=True)
inv_clean.reset_index(inplace=True)

In [24]:
rent_clean.to_csv('../data/interim/rent_clean.csv')
inv_clean.to_csv('../data/interim/inv_clean.csv')

#### Dataset 2: NYC COVID data

In [25]:
Apr = pd.read_csv("../data/raw/covidData/4.15Covid.csv").dropna()
May = pd.read_csv("../data/raw/covidData/5.16Covid.csv").dropna()
Jun = pd.read_csv("../data/raw/covidData/6.20Covid.csv").dropna() 
Jul = pd.read_csv("../data/raw/covidData/7.18Covid.csv").dropna() 
Aug = pd.read_csv("../data/raw/covidData/8.17Covid.csv").dropna() 
Sep = pd.read_csv("../data/raw/covidData/9.16Covid.csv").dropna() 
Oct = pd.read_csv("../data/raw/covidData/10.18Covid.csv").dropna()
Nov = pd.read_csv("../data/raw/covidData/11.16Covid.csv").dropna() 
May = May.astype({'MODIFIED_ZCTA': 'int64'})
Apr = Apr.astype({'MODIFIED_ZCTA': 'int64'})
Arr = [Apr,May,Jun,Jul,Aug,Sep,Oct,Nov]
for df in Arr:
    df.index=df["MODIFIED_ZCTA"]
    df = df.drop(["MODIFIED_ZCTA"],axis=1)

covidData = pd.DataFrame(index = [Apr["MODIFIED_ZCTA"]],
      columns=["Apr-Positive","Apr-Total Tests","Apr-Percent Positive","Apr-Increase of Pos","Apr-Rate of Increase",
               "May-Positive","May-Total Tests","May-Percent Positive","May-Increase of Pos","May-Rate of Increase",
               "Jun-Positive","Jun-Total Tests","Jun-Percent Positive","Jun-Increase of Pos","Jun-Rate of Increase",
               "Jul-Positive","Jul-Total Tests","Jul-Percent Positive","Jul-Increase of Pos","Jul-Rate of Increase",
               "Aug-Positive","Aug-Total Tests","Aug-Percent Positive","Aug-Increase of Pos","Aug-Rate of Increase",
               "Sep-Positive","Sep-Total Tests","Sep-Percent Positive","Sep-Increase of Pos","Sep-Rate of Increase",
               "Oct-Positive","Oct-Total Tests","Oct-Percent Positive","Oct-Increase of Pos","Oct-Rate of Increase",
               "Nov-Positive","Nov-Total Tests","Nov-Percent Positive","Nov-Increase of Pos","Nov-Rate of Increase",
               ])

months = ["Apr","May","Jun","Jul","Aug","Sep","Oct","Nov"]

for zipcode in Apr["MODIFIED_ZCTA"]:
    for month in months:
        if month == "Apr" or month == "May":
            covidData.loc[zipcode,month+"-Positive"]=Arr[months.index(month)].loc[zipcode,"Positive"]
            covidData.loc[zipcode,month+"-Total Tests"]=Arr[months.index(month)].loc[zipcode,"Total"]
            covidData.loc[zipcode,month+"-Percent Positive"]=Arr[months.index(month)].loc[zipcode,"zcta_cum.perc_pos"]

        else: 
            covidData.loc[zipcode,month+"-Positive"]=Arr[months.index(month)].loc[zipcode,"COVID_CASE_COUNT"]
            covidData.loc[zipcode,month+"-Total Tests"]=Arr[months.index(month)].loc[zipcode,"TOTAL_COVID_TESTS"]
            covidData.loc[zipcode,month+"-Percent Positive"]=Arr[months.index(month)].loc[zipcode,"PERCENT_POSITIVE"]

        if month !="Apr":
            prevMonth = months[months.index(month)-1]
            curPos=covidData.loc[zipcode,month+"-Positive"]
            prevPos=covidData.loc[zipcode,prevMonth+"-Positive"]
            covidData.loc[zipcode,month+"-Increase of Pos"]=curPos-prevPos
            rate = (curPos-prevPos)/prevPos
            covidData.loc[zipcode,month+"-Rate of Increase"]=rate


idx = list(covidData.index.values)
idx1 = [None]*(len(idx))
for i in range(len(idx)):
    idx1[i] = idx[i][0]
covidData.insert(0, 'zipcode', idx1)

covidData

,zipcode,Apr-Positive,Apr-Total Tests,Apr-Percent Positive,Apr-Increase of Pos,Apr-Rate of Increase,May-Positive,May-Total Tests,May-Percent Positive,May-Increase of Pos,...,Oct-Positive,Oct-Total Tests,Oct-Percent Positive,Oct-Increase of Pos,Oct-Rate of Increase,Nov-Positive,Nov-Total Tests,Nov-Percent Positive,Nov-Increase of Pos,Nov-Rate of Increase
MODIFIED_ZCTA,,,,,,,,,,,,,,,,,,,,,
10001,10001,238,502,47.41,NaN,NaN,329,1314,25.04,91,...,466,9739,4.78,11,0.0241758,530,11362,4.11,64,0.137339
10002,10002,645,1203,53.62,NaN,NaN,965,2766,34.89,320,...,1421,23960,5.93,69,0.0510355,1583,27538,5.71,162,0.114004
10003,10003,323,759,42.56,NaN,NaN,425,1841,23.09,102,...,643,23673,2.72,73,0.12807,802,28339,2.75,159,0.247278
10004,10004,23,60,38.33,NaN,NaN,28,117,23.93,5,...,54,1272,4.25,11,0.255814,61,1692,3.49,7,0.12963
10005,10005,41,126,32.54,NaN,NaN,56,315,17.78,15,...,119,3063,3.89,16,0.15534,132,3574,3.44,13,0.109244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11691,11691,1302,1904,68.38,NaN,NaN,2111,4736,44.57,809,...,3108,22353,13.9,319,0.114378,3270,25539,12.55,162,0.0521236
11692,11692,343,540,63.52,NaN,NaN,554,1322,41.91,211,...,738,6415,11.5,23,0.0321678,767,7308,10.43,29,0.0392954
11693,11693,228,384,59.38,NaN,NaN,309,768,40.23,81,...,366,3312,11.05,13,0.0368272,406,3918,10.03,40,0.10929


In [26]:
covid_num = covidData.iloc[:,[0,1,6,11,16,21,26,31,36]]
covid_num

,zipcode,Apr-Positive,May-Positive,Jun-Positive,Jul-Positive,Aug-Positive,Sep-Positive,Oct-Positive,Nov-Positive
MODIFIED_ZCTA,,,,,,,,,
10001,10001,238,329,381,410,433,455,466,530
10002,10002,645,965,1077,1199,1305,1352,1421,1583
10003,10003,323,425,458,497,525,570,643,802
10004,10004,23,28,32,36,39,43,54,61
10005,10005,41,56,64,75,91,103,119,132
...,...,...,...,...,...,...,...,...,...
11691,11691,1302,2111,2422,2535,2629,2789,3108,3270
11692,11692,343,554,629,664,693,715,738,767
11693,11693,228,309,333,344,349,353,366,406


In [27]:
covidData.to_csv('../data/interim/covidData.csv')
covid_num.to_csv('../data/interim/covid_num.csv')

#### Dataset 3: Census Data

In [28]:
census_data = pd.read_csv('../data/raw/acs/acs_nyc_data.csv',index_col=0)
census_data.columns = census_data.columns.str.lower().str.replace('%','_').str.replace(' ','_')

In [29]:
relevant_cols = [census_data.iloc[:,1]
                 ,census_data.iloc[:,[8,9,33,275,306,369,370,371,373]] #demographic #maybe add total by age group\
                 ,census_data.iloc[:,[642,643,649,652,655,656]], pd.to_numeric(census_data.iloc[:,822], errors='coerce').fillna(0)/census_data.iloc[:,783].values #commute and average commute time \
                 ,census_data.iloc[:,[1080]] #household income \
                 ,census_data.iloc[:,[1314]]/census_data.iloc[:,[1311]].values #unemployment rate
                 ,census_data.iloc[:,[1403,1748,1902,2127,2128,2129,2101,2185,2249,2370,2371,2372,2763,2765]],  census_data.iloc[:,1587:1591]#housing
                 ,census_data.iloc[:,[1409]]/census_data.iloc[:,[1407]].values] #% of units renter occupied

rel_census_data = pd.concat(relevant_cols,axis=1)

#fill na with mean
for col in rel_census_data.columns:
    if rel_census_data[col].dtypes == 'O':
        numeric_col = rel_census_data[col].apply(pd.to_numeric, errors='coerce').fillna(0)
        numeric_col_mean = np.mean(numeric_col)
        rel_census_data[col] = numeric_col.fillna(numeric_col_mean)

rel_census_data.head()

,zip_code_tabulation_area_(5-digit),sex_by_age__total_population_total,sex_by_age_total_population_total_male,sex_by_age_total_population_total_female,"sex_by_age_(white_alone,_not_hispanic_or_latino)__white_alone,_not_hispanic_or_latino_population_total",sex_by_age_(hispanic_or_latino)__hispanic_or_latino_population_total,race__total_population_total,race_total_population_total_white_alone,race_total_population_total_black_or_african_american_alone,race_total_population_total_asian_alone,...,lower_value_quartile_(dollars)__owner-occupied_housing_units_lower_value_quartile_(dollars),median_value_(dollars)__owner-occupied_housing_units_median_value_(dollars),upper_value_quartile_(dollars)__owner-occupied_housing_units_upper_value_quartile_(dollars),median_household_income_the_past_12_months_(in_2018_inflation-adjusted_dollars)_by_tenure_occupied_housing_units_median_household_income_in_the_past_12_months_(in_2018_inflation-adjusted_dollars)_total,median_household_income_the_past_12_months_(in_2018_inflation-adjusted_dollars)_by_tenure_occupied_housing_units_median_household_income_in_the_past_12_months_(in_2018_inflation-adjusted_dollars)_renter_occupied_(dollars),tenure_by_occupants_per_room_occupied_housing_units_total_renter_occupied_0.50_or_less_occupants_per_room,tenure_by_occupants_per_room_occupied_housing_units_total_renter_occupied_0.51_to_1.00_occupants_per_room,tenure_by_occupants_per_room_occupied_housing_units_total_renter_occupied_1.01_to_1.50_occupants_per_room,tenure_by_occupants_per_room_occupied_housing_units_total_renter_occupied_1.51_to_2.00_occupants_per_room,tenure_occupied_housing_units_total_renter_occupied
0,10001.0,22924,11671,11253,12182.0,3246.0,22924,14422,1413,5774,...,92400.0,343400.0,1559100.0,88526.0,83862.0,4740,4122,81,235,0.738316
1,10002.0,74993,36230,38763,16472.0,20390.0,74993,24168,6283,31089,...,480200.0,665200.0,945400.0,35859.0,31122.0,13295,13197,1513,872,0.866726
2,10003.0,54682,27503,27179,38521.0,4506.0,54682,41511,3178,7373,...,658800.0,1001200.0,1847500.0,112131.0,91081.0,9280,7749,121,364,0.670609
3,10004.0,3028,1267,1761,1834.0,112.0,3028,1902,51,922,...,655700.0,1640600.0,2000001.0,157645.0,146957.0,357,640,15,12,0.617239
4,10005.0,8831,4003,4828,6132.0,608.0,8831,6484,221,1497,...,652700.0,1415400.0,1880200.0,173333.0,180503.0,1029,1967,89,383,0.803612


In [30]:
rel_census_data.to_csv('../data/interim/rel_census_data.csv')